In [30]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import glob


file_list = glob.glob('asset/*.xlsx')

print(f'test file list : {file_list}')



for i in file_list:
    file_path = i.replace("\\","/")
    
# file data frame create
wb = load_workbook(filename=file_path,data_only=True)
ws = wb[wb.sheetnames[1]]
df = pd.DataFrame(ws.values)
tmp_df = df.rename(columns=df.iloc[1])
extract_df = tmp_df.drop(tmp_df.index[1]) 
extract_df.fillna('', inplace=True)

extract_df

# df = pd.read_excel(file_path, sheet_name=1)
# print(f'read_excel_file : {file_path}')
# tmp_df = df.rename(columns=df.iloc[0])
# extract_df = tmp_df.drop(tmp_df.index[0]) 
# extract_df.fillna('', inplace=True)
# extract_df.astype('string')



# extract_df['한글용어명']

test file list : ['asset\\표준화 작업-23.인사노무_V0.81.xlsx']


,No,미사용구분,DB명,테이블소유자,테이블명(영문),테이블명(한글),컬럼명(영문),컬럼명(한글),데이터 타입,데이터 길이,...,분류어chk,비고,시스템코드체크,분류어X,한단어,FLAG_여부X,EAI_FLAG_여부_체크,테이블컬럼중복,영문컬럼기준_한글용어다름,한글용어기준_영문용어다름
0,,,,,,,,,,,...,,,,,,,,,,
2,1,,인사노무,NOHR01,AL1001,(AL1001)진단회차관리,C_CD,회사코드,VARCHAR2,20,...,CD,,X,X,X,X,X,X,X,X
3,2,,인사노무,NOHR01,AL1001,(AL1001)진단회차관리,EVU_STD_ID,진단기준ID,VARCHAR2,20,...,ID,,X,X,X,X,X,X,O,O
4,3,,인사노무,NOHR01,AL1001,(AL1001)진단회차관리,EVU_STD_NM,진단기준명,VARCHAR2,100,...,NM,,X,X,X,X,X,X,O,O
5,4,,인사노무,NOHR01,AL1001,(AL1001)진단회차관리,EVU_YY,진단년도,VARCHAR2,4,...,YY,,X,X,X,X,X,X,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32912,32911,,인사노무,NOHR01,ZSAP_V_TBL_02,국외출장 출장비 내역,TRAF_UNIT,일비 단가,NUMBER,22,...,AMT,,X,X,X,X,X,X,X,X
32913,32912,,인사노무,NOHR01,ZSAP_V_TBL_02,국외출장 출장비 내역,TRAF_CNT,일비 일수,NUMBER,22,...,CNT,,X,X,X,X,X,X,O,O
32914,32913,,인사노무,NOHR01,ZSAP_V_TBL_02,국외출장 출장비 내역,TRAF_EXPN_SUM,일비,NUMBER,22,...,AMT,,X,X,X,X,X,X,X,X
32915,32914,,인사노무,NOHR01,ZSAP_V_TBL_02,국외출장 출장비 내역,BIZ_TRIP_EXPN_SUM,총금액,NUMBER,22,...,GRAMT,,X,X,X,X,X,X,O,O


In [31]:
extract_df['한글용어명']

0                
2        인사노무회사코드
3          진단기준ID
4           진단기준명
5            진단연도
           ...   
32912      일비단가금액
32913        일비일수
32914        일비금액
32915      출장비총금액
32916        환율내용
Name: 한글용어명, Length: 32916, dtype: object

In [32]:
extract_df['영문2']

0             
2           CO
3         CRTR
4         CRTR
5           YY
         ...  
32912    UNITC
32913       DD
32914      AMT
32915    GRAMT
32916      CTT
Name: 영문2, Length: 32916, dtype: object

In [43]:
def dup_encol (chk_col, col_en, filter_col_list):
    if chk_col =='X':
        if col_en in filter_col_list : 
            return 'O'
    else : 
        return 'O'

In [46]:
df_dup_chk2 =(~(extract_df.duplicated(['컬럼명(영문)','한글용어명'],keep=False)))&(extract_df['한글용어명']!='')&(extract_df['미사용구분']=='')&(extract_df.duplicated(['컬럼명(영문)'],keep=False))
extract_df['chk_col'] = np.where(df_dup_chk2, 'O', 'X')

filter_col_list = extract_df[extract_df['chk_col']=='O']['컬럼명(영문)'].to_list()

extract_df['chk_col_final'] = extract_df.apply(lambda row : dup_encol( row['chk_col'], row['컬럼명(영문)'],filter_col_list) , axis=1)



df1 = extract_df[extract_df['chk_col_final']=='O']

df2 = df1.sort_values(by=['컬럼명(영문)'], axis=0)

df2[['컬럼명(영문)','한글용어명']]

,컬럼명(영문),한글용어명
11587,ABLT_GRADE_11,역량11차등급명
10390,ABLT_GRADE_11,
10392,ABLT_GRADE_12,
11589,ABLT_GRADE_12,역량21차등급명
11685,ABLT_GRD_11,
...,...,...
9037,YYYY,
26029,ZCTRL_DEPT,인사노무예산운영부서코드
25572,ZCTRL_DEPT,인사노무예산운영부서코드
25476,ZCTRL_DEPT,인사노무예산운영부서코드
